In [1]:
import torch
import os
import numpy as np
import util
import data_util
import matplotlib.pyplot as plt
from glob import glob
from scipy.io import loadmat


from deep_voxels import DeepVoxels
from projection import ProjectionHelper

In [2]:
device = torch.device('cuda')
model_path = os.path.join('/home/max/cache/logs/', 'model-epoch_36_iter_24000.pth')
data_path = os.path.join('/home/max/classification_dataset/')

In [3]:
img_dim = 256
grid_dim = 32
proj_image_dims = [64, 64]

_, grid_barycenter, scale, near_plane, _ = \
    util.parse_intrinsics(os.path.join(data_path, 'intrinsics.txt'), trgt_sidelength=proj_image_dims[0])

if near_plane == 0.0:
    near_plane = np.sqrt(3)/2

In [4]:
lift_intrinsic = util.parse_intrinsics(os.path.join(data_path, 'intrinsics.txt'),
                                       trgt_sidelength=proj_image_dims[0])[0]
proj_intrinsic = lift_intrinsic

# Set up scale and world coordinates of voxel grid
voxel_size = (1. / grid_dim) * 1.1 * scale
grid_origin = torch.tensor(np.eye(4)).float().to(device).squeeze()
grid_origin[:3,3] = grid_barycenter

# Minimum and maximum depth used for rejecting voxels outside of the cmaera frustrum
depth_min = 0.
depth_max = grid_dim * voxel_size + near_plane
grid_dims = 3 * [grid_dim]

# Resolution of canonical viewing volume in the depth dimension, in number of voxels.
frustrum_depth = int(np.ceil(1.5 * grid_dim))

In [5]:
model = DeepVoxels(lifting_img_dims=proj_image_dims,
                   frustrum_img_dims=proj_image_dims,
                   grid_dims=grid_dims,
                   use_occlusion_net=True,
                   num_grid_feats=64,
                   nf0=64,
                   img_sidelength=img_dim)
model.to(device)


# Projection module
projection = ProjectionHelper(projection_intrinsic=proj_intrinsic,
                              lifting_intrinsic=lift_intrinsic,
                              depth_min=depth_min,
                              depth_max=depth_max,
                              projection_image_dims=proj_image_dims,
                              lifting_image_dims=proj_image_dims,
                              grid_dims=grid_dims,
                              voxel_size=voxel_size,
                              device=device,
                              frustrum_depth=frustrum_depth,
                              near_plane=near_plane)

OcclusionNet(
  (occlusion_prep): Sequential(
    (0): Conv3dSame(
      (net): Sequential(
        (0): ReplicationPad3d((1, 1, 1, 1, 1, 1))
        (1): Conv3d(65, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)
      )
    )
    (1): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (occlusion_net): Unet3d(
    (in_layer): Sequential(
      (0): Conv3dSame(
        (net): Sequential(
          (0): ReplicationPad3d((1, 1, 1, 1, 1, 1))
          (1): Conv3d(4, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)
        )
      )
      (1): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace)
    )
    (unet_block): UnetSkipConnectionBlock3d(
      (model): Sequential(
        (0): DownBlock3D(
          (net): Sequential(
            (0): ReplicationPad3d((1, 1, 1, 1, 1, 1))
            (1): Conv3d(4, 8, kernel_size=(4, 4, 4), stride=(2, 2,

In [6]:
util.custom_load(model, model_path)
model.eval()

DeepVoxels(
  (feature_extractor): Sequential(
    (0): DownsamplingNet(
      (downs): Sequential(
        (0): DownBlock(
          (net): Sequential(
            (0): ReflectionPad2d((1, 1, 1, 1))
            (1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
            (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): LeakyReLU(negative_slope=0.2, inplace)
            (4): ReflectionPad2d((1, 1, 1, 1))
            (5): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), bias=False)
            (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (7): LeakyReLU(negative_slope=0.2, inplace)
          )
        )
        (1): DownBlock(
          (net): Sequential(
            (0): ReflectionPad2d((1, 1, 1, 1))
            (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
            (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [7]:
train_rgb_list = sorted(glob(os.path.join(data_path,'train_rgb','*.png')))
train_pose_list = sorted(glob(os.path.join(data_path,'train_pose','*.mat')))
test_rgb_list = sorted(glob(os.path.join(data_path,'test_rgb','*.png')))
test_pose_list = sorted(glob(os.path.join(data_path,'test_pose','*.mat')))

In [8]:
label_str = ['bench','cabinet','car','chair','gun','light','phone','plane','screen','ship','sofa','stereo','table']
str2num = {label_str[i]:i for i in range(len(label_str))}

In [12]:
labels = list()
for i in range(len(train_rgb_list)//4):
    inpt_rgbs = list()
    backproj_mapping = list()
    for j in range(4):
        each_img = data_util.load_img(train_rgb_list[i*4+j], square_crop=True, downsampling_order=1, target_size=[img_dim, img_dim])
        each_img = each_img[:, :, :3].astype(np.float32) / 255. - 0.5
        each_img = each_img.transpose(2,0,1)
        each_img = torch.from_numpy(each_img)

        each_pose = loadmat(train_pose_list[i*4+j])['extrinsic']
        each_pose = np.linalg.inv(each_pose).astype(np.float32)
        each_pose = torch.from_numpy(each_pose)
        
        inpt_rgbs.append(each_img.unsqueeze(0).to(device))
        backproj_mapping.append(projection.comp_lifting_idcs(camera_to_world=each_pose.squeeze().to(device),grid2world=grid_origin))
        

    with torch.no_grad():
        model.representation = torch.zeros((1, model.n_grid_feats, model.grid_dims[0], model.grid_dims[1], model.grid_dims[2])).cuda()

        lift_volume_idcs, lift_img_coords = list(zip(*backproj_mapping))

        model(inpt_rgbs, 'pretest', None, None, lift_volume_idcs, lift_img_coords, None)
        repre = model.representation.detach().cpu().numpy()
        
    if i%10 == 0:
            print(i)
            
    np.save('/home/max/saved_data/train_repre/train_'+str(i).zfill(4), repre)
    label = str2num[train_rgb_list[4*i].split('/')[-1].split('_')[0]]
    labels.append(label)
    
np.save('/home/max/saved_data/train_repre/tr_labels', np.array(labels))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030


In [13]:
labels = list()
for i in range(len(test_rgb_list)//4):
    inpt_rgbs = list()
    backproj_mapping = list()
    for j in range(4):
        each_img = data_util.load_img(test_rgb_list[i*4+j], square_crop=True, downsampling_order=1, target_size=[img_dim, img_dim])
        each_img = each_img[:, :, :3].astype(np.float32) / 255. - 0.5
        each_img = each_img.transpose(2,0,1)
        each_img = torch.from_numpy(each_img)

        each_pose = loadmat(test_pose_list[i*4+j])['extrinsic']
        each_pose = np.linalg.inv(each_pose).astype(np.float32)
        each_pose = torch.from_numpy(each_pose)
        
        inpt_rgbs.append(each_img.unsqueeze(0).to(device))
        backproj_mapping.append(projection.comp_lifting_idcs(camera_to_world=each_pose.squeeze().to(device),grid2world=grid_origin))
        

    with torch.no_grad():
        model.representation = torch.zeros((1, model.n_grid_feats, model.grid_dims[0], model.grid_dims[1], model.grid_dims[2])).cuda()

        lift_volume_idcs, lift_img_coords = list(zip(*backproj_mapping))

        model(inpt_rgbs, 'pretest', None, None, lift_volume_idcs, lift_img_coords, None)
        repre = model.representation.detach().cpu().numpy()
        
    if i%10 == 0:
            print(i)
            
    np.save('/home/max/saved_data/test_repre/test_'+str(i).zfill(4), repre)
    label = str2num[test_rgb_list[4*i].split('/')[-1].split('_')[0]]
    labels.append(label)
    
np.save('/home/max/saved_data/test_repre/te_labels', np.array(labels))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
